# QAR数据分析系统 - 开发测试环境

这个Notebook用于本地开发和测试QAR系统的各个组件。

## 1. 环境设置

In [ ]:
import sys
import os
from pathlib import Path

# 添加项目根目录到Python路径
project_root = Path().absolute().parent
src_path = project_root / "src"
sys.path.insert(0, str(src_path))

print(f"项目根目录: {project_root}")
print(f"源代码路径: {src_path}")
print(f"Python版本: {sys.version}")

## 2. 检查依赖

In [ ]:
# 检查关键依赖
import torch
import pandas as pd
import numpy as np
import plotly
import fastapi
import sklearn

print("📦 依赖版本检查:")
print(f"PyTorch: {torch.__version__} (CPU: {not torch.cuda.is_available()})")
print(f"Pandas: {pd.__version__}")
print(f"NumPy: {np.__version__}")
print(f"Plotly: {plotly.__version__}")
print(f"FastAPI: {fastapi.__version__}")
print(f"Scikit-learn: {sklearn.__version__}")

## 3. 测试QAR模块导入

In [ ]:
# 测试QAR模块导入
try:
    from api.main import app
    print("✅ FastAPI应用导入成功")
except Exception as e:
    print(f"❌ FastAPI应用导入失败: {e}")

try:
    from models.qar_data_generator import QARDataGenerator
    print("✅ QAR数据生成器导入成功")
except Exception as e:
    print(f"❌ QAR数据生成器导入失败: {e}")

try:
    from visualization.qar_visualizer import QARVisualizer
    print("✅ QAR可视化器导入成功")
except Exception as e:
    print(f"❌ QAR可视化器导入失败: {e}")

## 4. 生成测试数据

In [ ]:
# 生成测试数据
from models.qar_data_generator import QARDataGenerator

generator = QARDataGenerator()
qar_data = generator.generate_flight_data(
    flight_duration_minutes=10,
    sampling_rate_hz=1.0,
    anomaly_rate=0.05
)

print(f"生成数据形状: {qar_data.shape}")
print(f"数据列: {list(qar_data.columns)}")
print("\n数据预览:")
qar_data.head()

## 5. 数据可视化测试

In [ ]:
# 测试数据可视化
from visualization.qar_visualizer import QARVisualizer

visualizer = QARVisualizer()

# 创建时间序列图
fig = visualizer.create_time_series_plot(qar_data)
fig.show()

print("✅ 时间序列图创建成功")

## 6. 异常检测测试

In [ ]:
# 测试异常检测
from models.anomaly_detector import AnomalyDetector

detector = AnomalyDetector()
anomalies = detector.detect_anomalies(qar_data)

print(f"检测到异常数量: {len(anomalies)}")
if len(anomalies) > 0:
    print("异常样本:")
    print(anomalies.head())

## 7. 性能分析

In [ ]:
import time

# 性能测试
print("🚀 性能测试:")

# 数据生成性能
start_time = time.time()
large_data = generator.generate_flight_data(
    flight_duration_minutes=60,
    sampling_rate_hz=1.0
)
generation_time = time.time() - start_time
print(f"生成60分钟数据耗时: {generation_time:.2f}秒")

# 可视化性能
start_time = time.time()
fig = visualizer.create_time_series_plot(large_data)
visualization_time = time.time() - start_time
print(f"创建可视化图表耗时: {visualization_time:.2f}秒")

# 异常检测性能
start_time = time.time()
anomalies = detector.detect_anomalies(large_data)
detection_time = time.time() - start_time
print(f"异常检测耗时: {detection_time:.2f}秒")

## 8. API测试

In [ ]:
import requests
import json

# 测试API (需要先启动服务器)
base_url = "http://127.0.0.1:8000"

try:
    # 健康检查
    response = requests.get(f"{base_url}/health", timeout=5)
    if response.status_code == 200:
        print("✅ API服务器连接正常")
        print(f"健康状态: {response.json()}")
    else:
        print(f"❌ API服务器响应异常: {response.status_code}")
except requests.exceptions.ConnectionError:
    print("❌ 无法连接到API服务器")
    print("💡 请先运行: python start_local.py")
except Exception as e:
    print(f"❌ API测试失败: {e}")

## 9. 开发工具

In [ ]:
# 开发辅助工具
def reload_modules():
    """重新加载模块"""
    import importlib
    modules_to_reload = [
        'models.qar_data_generator',
        'visualization.qar_visualizer',
        'models.anomaly_detector'
    ]
    
    for module_name in modules_to_reload:
        try:
            module = sys.modules.get(module_name)
            if module:
                importlib.reload(module)
                print(f"✅ 重新加载: {module_name}")
        except Exception as e:
            print(f"❌ 重新加载失败 {module_name}: {e}")

def clear_data():
    """清理数据变量"""
    global qar_data, large_data, anomalies
    del qar_data, large_data, anomalies
    print("🧹 数据变量已清理")

print("🛠️ 开发工具已加载")
print("使用 reload_modules() 重新加载模块")
print("使用 clear_data() 清理数据变量")